In [1]:
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.colors as mcolors
import os
from pathlib import Path

In [2]:
scenario = ['PAK', 'RUS', 'HOA', 'URU', 'ALL']
production_cap = False

current_dir = Path.cwd()
print(f"Current working directory:", current_dir)

loss_calculation_folder = current_dir / "evaluation"
print("input files at",loss_calculation_folder)

output_dir = os.path.join(
    loss_calculation_folder,
    "mit_cap" if production_cap else "ohne_cap")
print("output directory at", output_dir)


Current working directory: /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project
input files at /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project/evaluation
output directory at /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project/evaluation/ohne_cap


In [3]:
df = {}
for s in scenario:
    # Add conditional suffix to the filename based on production_cap
    filename_suffix = '_capped' if production_cap else '_no_cap'
    filename = loss_calculation_folder / f"{s}-highestLosses{filename_suffix}.csv"  
    
    if filename.exists():
        try:
            df[s] = pd.read_csv(filename)
            print(f"✓ Loaded: {filename}")
        except Exception as e:
            print(f"× Error loading {filename}: {str(e)}")
    else:
        print(f"! File not found: {filename}")

✓ Loaded: /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project/evaluation/PAK-highestLosses_no_cap.csv
✓ Loaded: /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project/evaluation/RUS-highestLosses_no_cap.csv
✓ Loaded: /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project/evaluation/HOA-highestLosses_no_cap.csv
✓ Loaded: /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project/evaluation/URU-highestLosses_no_cap.csv
✓ Loaded: /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project/evaluation/ALL-highestLosses_no_cap.csv


In [4]:
for s in scenario:  # Use scenario names directly instead of range(len())
    if s not in df:
        print(f"Skipping {s}: no data available")
        continue
        
    data = df[s]
    df_maize = data[data['item'] == 'Maize and products']
    
    # Group and sum losses by country
    maize_losses = df_maize.groupby('area')['al/capita [kg]'].sum().reset_index()
    
    # Handle edge cases
    if maize_losses.empty:
        print(f"Skipping {s}: no maize data available")
        continue

    # Clip outliers (1st and 99th percentiles)
    q01, q99 = maize_losses["al/capita [kg]"].quantile([0.01, 0.99])
    maize_losses["al/capita [kg]"] = maize_losses["al/capita [kg]"].clip(q01, q99)

    # Generate title and filename
    cap_status = "capped" if production_cap else "uncapped"
    title = f"Maize Losses per Capita [kg] - {s} ({'With' if production_cap else 'No'} Production Cap)"
    filename = f"maize_losses_per_capita_{s.lower()}_{cap_status}.png"
    plot_path = os.path.join(output_dir, filename)

    # Create choropleth map
    fig = px.choropleth(
        maize_losses,
        locations="area",
        locationmode="country names",
        color="al/capita [kg]",
        hover_name="area",
        color_continuous_scale=px.colors.diverging.Spectral[::-1],
        title=title,
        projection="natural earth"
    )

    # Format colorbar
    min_val = maize_losses["al/capita [kg]"].min()
    max_val = maize_losses["al/capita [kg]"].max()
    
    fig.update_layout(
        coloraxis_colorbar=dict(
            title="kg/capita",
            tickvals=np.linspace(min_val, max_val, 5),
            thickness=20,
            len=0.75
        ),
        margin=dict(l=0, r=0, t=50, b=0),
        font=dict(size=14),
        title_x=0.5
    )

    # Save and show
    fig.write_image(plot_path, scale=2, width=1200, height=800, engine="kaleido")
    print(f"Saved map: {plot_path}")
    fig.show()

Skipping PAK: no maize data available
Saved map: /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project/evaluation/ohne_cap/maize_losses_per_capita_rus_uncapped.png


Skipping HOA: no maize data available
Skipping URU: no maize data available
Saved map: /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project/evaluation/ohne_cap/maize_losses_per_capita_all_uncapped.png


In [5]:
for s in scenario:
    if s not in df:
        print(f"Skipping {s}: no data available")
        continue
        
    data = df[s]
    df_poultry = data[data['item'] == 'Poultry Birds']
    
    # Group and sum losses by country
    poultry_losses = df_poultry.groupby('area')['al/capita [kg]'].sum().reset_index()
    
    # Handle edge cases
    if poultry_losses.empty:
        print(f"Skipping {s}: no Poultry Birds data available")
        continue

    # Clip outliers (1st and 99th percentiles)
    q01, q99 = poultry_losses["al/capita [kg]"].quantile([0.01, 0.99])
    poultry_losses["al/capita [kg]"] = poultry_losses["al/capita [kg]"].clip(q01, q99)

    # Generate title and filename
    cap_status = "capped" if production_cap else "uncapped"
    title = f"Poultry Birds Losses per Capita [kg] - {s} ({'With' if production_cap else 'No'} Production Cap)"
    filename = f"poultry_losses_per_capita_{s.lower()}_{cap_status}.png"
    plot_path = os.path.join(output_dir, filename)

    # Get min and max values for legend
    min_val = poultry_losses["al/capita [kg]"].min()
    max_val = poultry_losses["al/capita [kg]"].max()

    # Create choropleth map with proper legend
    fig = px.choropleth(
        poultry_losses,
        locations="area",
        locationmode="country names",
        color="al/capita [kg]",
        hover_name="area",
        color_continuous_scale=px.colors.diverging.Spectral[::-1],
        range_color=[min_val, max_val],  # Explicitly set color range
        title=title,
        projection="natural earth",
        labels={'al/capita [kg]': 'Loss per capita (kg)'}  # Proper legend label
    )

    # Format layout with clear legend
    fig.update_layout(
        coloraxis=dict(
            colorbar=dict(
                title="kg/capita",
                tickvals=np.linspace(min_val, max_val, 5),
                thickness=20,
                len=0.75
            ),
        ),
        margin=dict(l=0, r=0, t=50, b=0),
        font=dict(size=14),
        title_x=0.5
    )

    # Save and show
    fig.write_image(plot_path, scale=2, width=1200, height=800, engine="kaleido")
    print(f"Saved map: {plot_path}")
    fig.show()

Saved map: /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project/evaluation/ohne_cap/poultry_losses_per_capita_pak_uncapped.png


Saved map: /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project/evaluation/ohne_cap/poultry_losses_per_capita_rus_uncapped.png


Saved map: /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project/evaluation/ohne_cap/poultry_losses_per_capita_hoa_uncapped.png


Saved map: /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project/evaluation/ohne_cap/poultry_losses_per_capita_uru_uncapped.png


Saved map: /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project/evaluation/ohne_cap/poultry_losses_per_capita_all_uncapped.png


In [6]:
for s in scenario:
    if s not in df:
        print(f"Skipping {s}: no data available")
        continue
        
    data = df[s]
    df_rice = data[data['item'] == 'Rice and products']
    
    # Group and sum losses by country
    rice_losses = df_rice.groupby('area')['al/capita [kg]'].sum().reset_index()
    
    # Handle edge cases
    if rice_losses.empty:
        print(f"Skipping {s}: no Rice and products data available")
        continue

    # Clip outliers (1st and 99th percentiles)
    q01, q99 = poultry_losses["al/capita [kg]"].quantile([0.01, 0.99])
    rice_losses["al/capita [kg]"] = rice_losses["al/capita [kg]"].clip(q01, q99)

    # Generate title and filename
    cap_status = "capped" if production_cap else "uncapped"
    title = f"Rice Losses per Capita [kg] - {s} ({'With' if production_cap else 'No'} Production Cap)"
    filename = f"rice_losses_per_capita_{s.lower()}_{cap_status}.png"
    plot_path = os.path.join(output_dir, filename)

    # Get min and max values for legend
    min_val = rice_losses["al/capita [kg]"].min()
    max_val = rice_losses["al/capita [kg]"].max()

    # Create choropleth map with proper legend
    fig = px.choropleth(
        rice_losses,
        locations="area",
        locationmode="country names",
        color="al/capita [kg]",
        hover_name="area",
        color_continuous_scale=px.colors.diverging.Spectral[::-1],
        range_color=[min_val, max_val],  # Explicitly set color range
        title=title,
        projection="natural earth",
        labels={'al/capita [kg]': 'Loss per capita (kg)'}  # Proper legend label
    )

    # Format layout with clear legend
    fig.update_layout(
        coloraxis=dict(
            colorbar=dict(
                title="kg/capita",
                tickvals=np.linspace(min_val, max_val, 5),
                thickness=20,
                len=0.75
            ),
        ),
        margin=dict(l=0, r=0, t=50, b=0),
        font=dict(size=14),
        title_x=0.5
    )

    # Save and show
    fig.write_image(plot_path, scale=2, width=1200, height=800, engine="kaleido")
    print(f"Saved map: {plot_path}")
    fig.show()

Saved map: /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project/evaluation/ohne_cap/rice_losses_per_capita_pak_uncapped.png


Skipping RUS: no Rice and products data available
Skipping HOA: no Rice and products data available
Skipping URU: no Rice and products data available
Saved map: /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project/evaluation/ohne_cap/rice_losses_per_capita_all_uncapped.png


In [7]:
for s in scenario:
    if s not in df:
        print(f"Skipping {s}: no data available")
        continue
        
    data = df[s]
    df_wheat = data[data['item'] == 'Wheat and products']
    
    # Group and sum losses by country
    wheat_losses = df_wheat.groupby('area')['al/capita [kg]'].sum().reset_index()
    
    # Handle edge cases
    if wheat_losses.empty:
        print(f"Skipping {s}: no Wheat and products data available")
        continue

    # Clip outliers (1st and 99th percentiles)
    q01, q99 = poultry_losses["al/capita [kg]"].quantile([0.01, 0.99])
    wheat_losses["al/capita [kg]"] = wheat_losses["al/capita [kg]"].clip(q01, q99)

    # Generate title and filename
    cap_status = "capped" if production_cap else "uncapped"
    title = f"Rice Losses per Capita [kg] - {s} ({'With' if production_cap else 'No'} Production Cap)"
    filename = f"rice_losses_per_capita_{s.lower()}_{cap_status}.png"
    plot_path = os.path.join(output_dir, filename)

    # Get min and max values for legend
    min_val = wheat_losses["al/capita [kg]"].min()
    max_val = wheat_losses["al/capita [kg]"].max()

    # Create choropleth map with proper legend
    fig = px.choropleth(
        wheat_losses,
        locations="area",
        locationmode="country names",
        color="al/capita [kg]",
        hover_name="area",
        color_continuous_scale=px.colors.diverging.Spectral[::-1],
        range_color=[min_val, max_val],  # Explicitly set color range
        title=title,
        projection="natural earth",
        labels={'al/capita [kg]': 'Loss per capita (kg)'}  # Proper legend label
    )

    # Format layout with clear legend
    fig.update_layout(
        coloraxis=dict(
            colorbar=dict(
                title="kg/capita",
                tickvals=np.linspace(min_val, max_val, 5),
                thickness=20,
                len=0.75
            ),
        ),
        margin=dict(l=0, r=0, t=50, b=0),
        font=dict(size=14),
        title_x=0.5
    )

    # Save and show
    fig.write_image(plot_path, scale=2, width=1200, height=800, engine="kaleido")
    print(f"Saved map: {plot_path}")
    fig.show()

Skipping PAK: no Wheat and products data available
Saved map: /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project/evaluation/ohne_cap/rice_losses_per_capita_rus_uncapped.png


Skipping HOA: no Wheat and products data available
Skipping URU: no Wheat and products data available
Saved map: /Users/valentinbast/uni_codeprojects/compelx_sys_modelling/project/code_project/evaluation/ohne_cap/rice_losses_per_capita_all_uncapped.png
